### Задание 7. Транспортная задача

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [2]:
C = np.array([list(map(int, input(f'Введите {i+1} строку матрицы C:\n').split())) for i in range(3)])
print('\nМатрица C:')
print(C)

Введите 1 строку матрицы C:
16 14 8 10 4
Введите 2 строку матрицы C:
13 11 12 6 5
Введите 3 строку матрицы C:
5 9 16 17 27

Матрица C:
[[16 14  8 10  4]
 [13 11 12  6  5]
 [ 5  9 16 17 27]]


In [3]:
a1, a2, a3 = map(int, input('Введите через пробел числа a1, a2, a3:\n').split())

Введите через пробел числа a1, a2, a3:
280 330 290


In [4]:
A = np.array([a1, a2, a3])

In [5]:
b1, b2, b3, b4, b5 = map(int, input('Введите через пробел числа b1, b2, b3, b4, b5:\n').split())

Введите через пробел числа b1, b2, b3, b4, b5:
150 120 210 270 150


In [6]:
B = np.array([b1, b2, b3, b4, b5])

In [27]:
N = len(A)
M = len(B)

In [28]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'aᵢ \\ bⱼ'
df

aᵢ \ bⱼ,150,120,210,270,150
280,16,14,8,10,4
330,13,11,12,6,5
290,5,9,16,17,27


In [29]:
if sum(A) == sum(B):
    print('Задача закрытого типа')
else:
    print('Задача открытого типа')

Задача закрытого типа


Допустимый план перевозки груза методом минимальной стоимости

In [58]:
P = np.zeros_like(C)
INF = 1000000
C_copy = C.copy()
A_copy = A.copy()
B_copy = B.copy()
i = 0
j = 0

In [65]:
while not np.all(C_copy == INF):
    i, j = np.unravel_index(C_copy.argmin(), C_copy.shape)
    P[i][j] = min(A_copy[i], B_copy[j])
    if A_copy[i] <= B_copy[j]:
        C_copy[i].fill(INF)
        B_copy[j] -= A_copy[i]
        A_copy[i] = 0
    else:
        C_copy[:, j].fill(INF)
        A_copy[i] -= B[j]
        B_copy[j] = 0  

[[1000000 1000000 1000000 1000000 1000000]
 [1000000 1000000 1000000 1000000 1000000]
 [1000000 1000000 1000000 1000000 1000000]]


In [40]:
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план является невырожденным')
else:
    print('Опорный план является вырожденным')

Опорный план является невырожденным


In [41]:
F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана', F)

Полученный план перевозки груза:
 [[  0   0 130   0 150]
 [  0   0  60 270   0]
 [150 120  20   0   0]]
Значение целевой функции для этого опорного плана 6130


Оптимальный план перевозки груза

In [42]:
#A_u - матрица коэф. при x1,x2,...,xn в ограничениях, A_u @ x <= b_b (E @ x <= A)
#c0 - коэффициенты перед переменными в целевой функции
A_u = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_u = A

In [43]:
#A_ub - матрица коэф. при x1,x2,...,xn в ограничениях, A_eq @ x = b_eq (E @ x = B)
#c0 - коэффициенты перед переменными в целевой функции
b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

In [44]:
res = linprog(C.flatten(), A_u, b_u, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

res

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 173.0               
0.1306327387553     0.1306327387553     0.1306327387553     0.8788192907317  0.1306327387553     1100.449313969      
0.03181897250588    0.03181897250588    0.03181897250587    0.7810570896456  0.03181897250585    2466.047908114      
0.005361274551224   0.005361274551238   0.005361274551236   0.8472010400165  0.005361274551011   4593.444823824      
0.001066170730798   0.001066170730801   0.0010661707308     0.8055292353227  0.001066170730756   5622.438688044      
2.871636126877e-06  2.871636126411e-06  2.871636126702e-06  0.9987584128955  2.871636118785e-06  5949.02169793       
1.436143252222e-10  1.436143136588e-10  1.436143260508e-10  0.99994998868    1.436148148581e-10  5949.999951066      
Optimization terminated successfully.
         Current fu

     con: array([0.0000013 , 0.00000104, 0.00000183, 0.00000236, 0.0000013 ])
     fun: 5949.999951066457
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([0.00000243, 0.00000287, 0.00000252])
  status: 0
 success: True
       x: array([  0.00000002,   0.00000004, 189.99999815,   0.00000004,
        89.99999931,   0.00000003,   0.00000005,   0.0000001 ,
       269.99999759,  59.99999936, 149.99999865, 119.99999887,
        19.99999991,   0.00000001,   0.00000003])

In [45]:
print('Оптимальный план перевозки груза X*:\n', X)
print('Оптимальная стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза X*:
 [[  0.   0. 190.   0.  90.]
 [  0.   0.   0. 270.  60.]
 [150. 120.  20.   0.   0.]]
Оптимальная стоимость перевозки:
F_min = F(X*) = 5950.0
